# Basic integration of ODEs tutorial

This is a tutorial showing how to integreate ODEs using a handful of functions I placed in a module called "bcodes", for Biochemical ODEs. In this tutorial I use bcodes to create the inputs for scipy's solve_ivp API, which performs the integration of the ODEs.

A simple system of ODEs is used for illustration. The model consist of two dynamic mass balances, one for glucose and the other for biomass dry weight. 

$\dot{glc} = - q_{glc} \cdot X$

$\dot{X} = q_{glc} \cdot Y_{xs} \cdot X $

Where

$q_{glc} = \frac{q_{glc}^{max} \cdot \frac{n_{glc}}{bw}}{Km_{q\_glc} + \frac{n_{glc}}{bw}}$ 

## Parameters

The parameters are taken from bakers yeast growing aerobically on glucose. 

| Parameter | Units | Value | Comment |
|---|---|---|---|
|$Y_{xs}$ | $\frac{mmol_{glc}}{gDW}$ | 0.09 | Calculated from 0.5 g/g|
| $Km_{q\_glc}$ | mmol/kg | 1 | |
| $q_{glc}^{max}$ | mmol / (gDW h) | 3.89 | yields a $\mu^{max} = 0.35 h^{-1}$ |
| bw | kg | 1 | broth weight (fixed) |



In [ ]:
import os
import sys
root_path = os.path.split(os.getcwd())[0]
if root_path not in sys.path:
    sys.path.append(root_path)
    
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:
from ratevector import create_rate_vector
from stoichiometrymatrix import build_stoichiometry_matrix

# Model components

In [ ]:
id_sp = [
        'glc',  # mmol
        'dw'  # g DW
        ]

id_rs = [
        'v_glc',  # mmol/h  # glucose consumption rate (q_glc * dw)
        ]

rates = {
        'v_glc': 'dw * q_max_glc * (glc/bw) / (Km_q_glc + (glc/bw))',
        }

params = {
        'Yxs': 0.09,  # mmol_glc/gDW  # calculated from 0.5 g/g
        'Km_q_glc': 1,  # mmol/kg_bw
        'q_max_glc': 3.89,  # mmol/(gDW h)  # calcuated for mu_max = 0.35 1/h
        'bw': 1,  # kg  # broth weight
        }

mass_balances = {
        'glc': {'v_glc': -1},
        'dw': {'v_glc': params['Yxs']}
        }

init = [100, 0.2]

## Basic case

Integrating ODEs with all parameters substituted in the rate equations

In [ ]:
S = build_stoichiometry_matrix(id_sp, id_rs, mass_balances)

v_01 = create_rate_vector(id_sp, id_rs, rates, params)

def odes_01(t, y):
    return np.dot(S, v_01(y))

sol_01 = solve_ivp(odes_01, [0, 9], init, method='LSODA', vectorized=True,
        dense_output=True)

vt_01 = np.array(v_01(sol_01.y)).T

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(10, 8))
ax[0, 0].plot(sol_01.t, sol_01.y.T[:, 0])
ax[0, 0].set_ylabel('Glucose (mmol)')
ax[0, 1].plot(sol_01.t, sol_01.y.T[:, 1])
ax[0, 1].set_ylabel('Dry weight (g)')
ax[1, 0].plot(sol_01.t, vt_01[:, 0]/sol_01.y.T[:, 1])
ax[1, 0].set_ylabel('q_glc [mmol / (gDW h)]')
ax[1, 1].plot(sol_01.t, vt_01[:, 0] * params['Yxs']/sol_01.y.T[:, 1])
ax[1, 1].set_ylabel('$\mu\,(h^{-1})$')
ax[1, 0].set_xlabel('Time (h)')
ax[1, 1].set_xlabel('Time (h)')
fig.tight_layout();

## Adding parameters as arguments

In [ ]:
# Here substitute Yxs and q_max_glc by elements of a parameter vector p
params_02 = params.copy()
params_02['Yxs'] = 'p[0]'
params_02['q_max_glc'] = 'p[1]'

my_p = [0.1, 4.0]

v_02 = create_rate_vector(id_sp, id_rs, rates, params_02, p_is_arg=True)

def odes_02(t, y, p):
    return np.dot(S, v_02(y, p))

# Note. To pass arguments to solve_ivp, you need to use a lambda function. 
# You cannot pass arguments as a tuple as you could with the old odeint API
sol_02 = solve_ivp(
    lambda t, y: odes_02(t, y, my_p),
    [0, 9], init, method='LSODA', vectorized=True
)

## Adding a function that depends on time

Imagine the yield decreases as the fermentation proceeds. For example, as the cells grow, a by-product accumulates and inhibits the glucose transporter.

In [ ]:
k = np.log(0.8)/8
f_qmax_t = lambda t: params['q_max_glc'] * np.exp(k * t)
plt.plot(np.linspace(0, 9), f_qmax_t(np.linspace(0, 9)))
plt.xlabel('Time (h)')
plt.ylabel('q_max [mmol/(gDW h)')
plt.ylim(0, 4);

In [ ]:
params_03 = params.copy()
params_03['q_max_glc'] = 'f_qmax_t(t)'

In [ ]:
v_03 = create_rate_vector(id_sp, id_rs, rates, params_03,
                          t_is_arg=True, eval_scope={'f_qmax_t': f_qmax_t})
def odes_03(t, y):
    return np.dot(S, v_03(y, t))

sol_03 = solve_ivp(odes_03, [0, 9], init, method='LSODA', vectorized=True)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 4))
ax[0].plot(sol_01.t, sol_01.y.T[:, 0])
ax[0].plot(sol_03.t, sol_03.y.T[:, 0])
ax[0].set_ylabel('Glucose (mmol)')
ax[1].plot(sol_01.t, sol_01.y.T[:, 1])
ax[1].plot(sol_03.t, sol_03.y.T[:, 1])
ax[1].set_ylabel('Dry weight (g)')
ax[0].set_xlabel('Time (h)')
ax[1].set_xlabel('Time (h)')
fig.tight_layout();